<a href="https://colab.research.google.com/github/pachterlab/BGP_2023/blob/main/analysis/snATAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
start_time = time.time()

In [1]:
!git clone https://github.com/pachterlab/BGP_2023.git

Cloning into 'BGP_2023'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 110 (delta 18), reused 38 (delta 8), pack-reused 47
Receiving objects: 100% (110/110), 233.43 MiB | 20.05 MiB/s, done.
Resolving deltas: 100% (23/23), done.
Updating files: 100% (43/43), done.


In [2]:
# kb python for quantification
!pip install --quiet kb-python

# install minimap for mapping paired end reads
!wget --quiet --show-progress "https://github.com/lh3/minimap2/releases/download/v2.24/minimap2-2.24_x64-linux.tar.bz2"
!tar -jxf minimap2-2.24_x64-linux.tar.bz2
!cp minimap2-2.24_x64-linux/minimap2 /usr/local/bin

# Insstall samtools for sam file manipulation
# !wget --quiet --show-progress https://github.com/biod/sambamba/releases/download/v0.8.2/sambamba-0.8.2-linux-amd64-static.gz
# !gunzip sambamba-0.8.2-linux-amd64-static.gz
# !chmod +x /content/sambamba-0.8.2-linux-amd64-static
# !mv sambamba-0.8.2-linux-amd64-static /usr/local/bin/sambamba

!wget --quiet --show-progress https://github.com/samtools/samtools/releases/download/1.17/samtools-1.17.tar.bz2
!tar -jxf samtools-1.17.tar.bz2
!cd samtools-1.17/ && ./configure -q --prefix=$(pwd) && make -s && make install
!cp samtools-1.17/samtools /usr/local/bin

# Genrich for peak callling
!git clone https://github.com/jsh58/Genrich.git
!cd Genrich && make && cp Genrich /usr/local/bin

# bedtools for slicing the peaks
!wget --quiet --show-progress https://github.com/arq5x/bedtools2/releases/download/v2.30.0/bedtools.static.binary
!chmod +x /content/bedtools.static.binary
!mv /content/bedtools.static.binary /usr/local/bin/bedtools

!git clone --quiet https://github.com/pachterlab/kallisto && cd kallisto && git checkout shareseq
!apt-get -qq install autoconf
!cd kallisto && mkdir build && cd build && cmake .. && make &> /dev/null
!cp kallisto/build/src/kallisto $(pip show kb-python | grep "Location" | cut -f2 -d":")/kb_python/bins/linux/kallisto/kallisto
!cp kallisto/build/src/kallisto /usr/local/bin
!cp $(pip show kb-python | grep "Location" | cut -f2 -d":")/kb_python/bins/linux/bustools/bustools /usr/local/bin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 59.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 6.1 MB/s eta 0:00:00
minimap2-2.24_x64-l 100%[=========

In [14]:
!kallisto version

kallisto, version 0.48.0


In [15]:
!minimap2 --version

2.24-r1122


In [11]:
!samtools version

samtools 1.17
Using htslib 1.17
Copyright (C) 2023 Genome Research Ltd.

Samtools compilation details:
    Features:       build=configure curses=yes 
    CC:             gcc
    CPPFLAGS:       
    CFLAGS:         -Wall -g -O2
    LDFLAGS:        
    HTSDIR:         htslib-1.17
    LIBS:           
    CURSES_LIB:     -lncursesw

HTSlib compilation details:
    Features:       build=configure libcurl=yes S3=yes GCS=yes libdeflate=yes lzma=yes bzip2=yes plugins=no htscodecs=1.4.0
    CC:             gcc
    CPPFLAGS:       
    CFLAGS:         -Wall -g -O2 -fvisibility=hidden
    LDFLAGS:        -fvisibility=hidden 

HTSlib URL scheme handlers present:
    built-in:	 preload, data, file
    S3 Multipart Upload:	 s3w, s3w+https, s3w+http
    Amazon S3:	 s3+https, s3+http, s3
    Google Cloud Storage:	 gs+http, gs+https, gs
    libcurl:	 imaps, pop3, http, smb, gopher, sftp, ftps, imap, smtp, smtps, rtsp, scp, ftp, telnet, rtmp, ldap, https, ldaps, smbs, tftp, pop3s, dict
    crypt4gh-

In [17]:
!Genrich --version

Genrich, version 0.6.2
Copyright (C) 2018 John M. Gaspar (jsh58@wildcats.unh.edu)


In [19]:
!bedtools --version

bedtools v2.30.0


In [28]:
!wget --quiet --show-progress "http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz"

Homo_sapiens.GRCh38 100%[+++================>] 840.39M   776KB/s    in 15m 45s 


In [29]:
!time minimap2 \
-d ref.mmi \
Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

/bin/bash: line 1: 11382 Killed                  minimap2 -d ref.mmi Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

real	2m2.635s
user	2m40.966s
sys	0m25.301s


In [ ]:
!gunzip Homo_sapiens.GRCh38.dna.primary_assembly.fa \
&& fold -w 80 Homo_sapiens.GRCh38.dna.primary_assembly.fa > genome.fa

In [27]:
!BGP_2023/analysis/snATAK_ref.sh \
-o ref -g genome.fa -m ref.mmi \
-c BGP_2023/data/example/CB.fastq.gz \
-1 BGP_2023/data/example/R1.fastq.gz \
-2 BGP_2023/data/example/R2.fastq.gz 

Output: 
Genome: 
MMI: 
CB fastq: 
R1 fastq: 
R2 fastq: 
Error
Usage: /content/BGP_2023/analysis/snATAK_ref.sh [options]
    
    Options:
    -o, --output
    -g, --genome
    -m, --mmi
    -c, --cbfastq
    -1, --r1fastq
    -2, --r2fastq
    


In [ ]:
!ls -lht peaks.idx

-rw-r--r-- 1 root root 572M Oct  7 23:50 peaks.idx


In [ ]:
!head t2g.txt

1:10015-10460	1:10015-10460	1:10015-10460
1:11154-11266	1:11154-11266	1:11154-11266
1:29210-29373	1:29210-29373	1:29210-29373
1:29480-29798	1:29480-29798	1:29480-29798
1:99502-99868	1:99502-99868	1:99502-99868
1:104185-104255	1:104185-104255	1:104185-104255
1:143396-143517	1:143396-143517	1:143396-143517
1:149332-149519	1:149332-149519	1:149332-149519
1:180546-180929	1:180546-180929	1:180546-180929
1:181290-181834	1:181290-181834	1:181290-181834


In [3]:
!zcat BGP_2023/data/example/R1.fastq.gz | head -2
!zcat BGP_2023/data/example/R2.fastq.gz | head -2
!zcat BGP_2023/data/example/CB.fastq.gz | head -2

@A00836:480:HJ2W5DSXY:1:1101:30355:1016 1:N:0:CCGAGAAC
CNCCGGACCCACTCCCATTACCGTCTGGATCAGCAGCTCCAGGTCGCGGG
@A00836:480:HJ2W5DSXY:1:1101:30355:1016 3:N:0:CCGAGAAC
CCCTAGTGCTGTCCGTGACTTCTTTGGATCATGTCTGGCCCAAGATGGG
@A00836:480:HJ2W5DSXY:1:1101:30355:1016 2:N:0:CCGAGAAC
CCGTAAACACCAAAGC


In [ ]:
!kb count \
-i peaks.idx \
-g t2g.txt \
-x 10xATAC \
-o out \
-w 737K-cratac-v1.txt \
-t 2 \
--h5ad \
CB.fastq.gz R1.fastq.gz R2.fastq.gz

[2022-10-07 23:50:56,040]    INFO [count] Using index peaks.idx to generate BUS file to out from
[2022-10-07 23:50:56,040]    INFO [count]         min-R2.fastq.gz
[2022-10-07 23:50:56,041]    INFO [count]         min-R1.fastq.gz
[2022-10-07 23:50:56,041]    INFO [count]         min-R3.fastq.gz
[2022-10-07 23:51:02,971]    INFO [count] Sorting BUS file out/output.bus to out/tmp/output.s.bus
[2022-10-07 23:51:08,397]    INFO [count] Inspecting BUS file out/tmp/output.s.bus
[2022-10-07 23:51:10,008]    INFO [count] Correcting BUS records in out/tmp/output.s.bus to out/tmp/output.s.c.bus with whitelist 737K-cratac-v1.txt
[2022-10-07 23:51:13,032]    INFO [count] Sorting BUS file out/tmp/output.s.c.bus to out/output.unfiltered.bus
[2022-10-07 23:51:18,060]    INFO [count] Generating count matrix out/counts_unfiltered/cells_x_genes from BUS file out/output.unfiltered.bus
[2022-10-07 23:51:20,075]    INFO [count] Reading matrix out/counts_unfiltered/cells_x_genes.mtx
[2022-10-07 23:51:21,075]

In [ ]:
# Since we are doing read counting we need to use `--cm`, once this is implemented in kb we can just do it there
!bustools count \
-o out/counts_unfiltered/cells_x_genes \
-g t2g.txt \
-e out/matrix.ec \
-t out/transcripts.txt \
--genecounts \
--cm \
out/output.unfiltered.bus

In [ ]:
(time.time()-start_time)/60

11.038964438438416